#  Using FABRIC NVMe Devices

Your compute nodes can include fast NVMe storage devices. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use FABRIC NVMe storage. 


## Configure the Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_BASTION_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

# Set the keypair FABRIC will install in your slice.
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create a Node

The cell below creates a slice that contains a single node with a 1TB NVMe device. 


### Set the Slice Name and FABRIC Site

In [ ]:
from fabrictestbed.slice_editor import ComponentModelType

slice_name="MySliceNVME"
site="MAX"
node_name='Node1'
username='centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 100

nvme_name='nvme1'

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Add node
    node = slice.add_node(name=node_name, site=site)
    node.set_capacities(cores=cores, ram=ram, disk=disk)
    node.set_image(image, username)
    
    #Add an NVME Drive
    node.add_component(model='NVME_P4510', name=nvme_name)

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")

## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Slice: {slice.get_name()}")
except Exception as e:
    print(f"Get Slice Error: {e}")

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
try:
    node = slice.get_node(node_name) 
    print(f"Node Name        : {node.get_name()}")
    print(f"Management IP    : {node.get_management_ip()}")
    print(f"SSH Command      : {node.get_ssh_command()}")
    print()

    nvme1 = node.get_component(nvme_name)
    print(f"NVMe Name        : {nvme1.get_name()}")
    print(f"Details          : {nvme1.get_details()}")
    print(f"Disk (G)         : {nvme1.get_disk()}")
    print(f"Units            : {nvme1.get_unit()}")
    print(f"PCI Address      : {nvme1.get_pci_addr()}")
    print(f"Model            : {nvme1.get_model()}")
    print(f"Type             : {nvme1.get_type()}")  
except Exception as e:
    print(f"Error: {e}")

## Configure the NVMe PCI Device

NVMe storage is provided as bare PCI block devices and will likely need to be partitioned, formated, and mounted before use.

In [ ]:
try:
    nvme1.configure_nvme()
except Exception as e:
    print(f"Error: {e}")

## Cleanup Your Experiment

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")